In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/LICENSE
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/README.md
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/requirements.txt
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/.gitattributes
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Gas Sensors Measurements/Gas_Sensors_Measurements.csv
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images/Smoke/1266_Smoke.png
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images/Smoke/1443_Smoke.png
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images/Smoke/666_Smoke.png
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images/Smoke/1367_Smoke.png
/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images/Smoke/273_Smoke.png
/kagg

In [3]:
import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Add the 'src' folder to Python path
sys.path.append("/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/src")

from models import build_sensor_model  # Only sensor model
from data_loader import load_and_split_sensor_data  # Sensor data loader

# File paths and config
CSV_PATH = "/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Gas Sensors Measurements/Gas_Sensors_Measurements.csv"
BATCH_SIZE = 32
EPOCHS = 30
MODEL_SAVE_PATH = "saved_models/best_sensor_model.keras"

# Create model save directory
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# Load the sensor dataset
train_sen_ds, val_sen_ds, test_sen_ds = load_and_split_sensor_data(
    csv_path=CSV_PATH,
    test_size=0.1,
    val_size=0.2,
    random_state=42
)


2025-05-05 19:00:04.753229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746471604.907874      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746471604.955086      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1746471619.643561      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746471619.644297      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

In [4]:
from models import build_sensor_model, MCDropout, RandomSensorDropout  # Import necessary components
#dropout=0.036, hidden=224,240, lr=7.07e-04
# Build the sensor-only model
sensor_model = build_sensor_model(
    input_dim=7,               # Adjust if feature count differs
    sensor_dropout_rate=0.3,
    layer_dropout=0.036,
    hidden_units=(224,240),
    output_units=4,            # Number of gas classes
    lr=7.07e-04
)

# Display model architecture
sensor_model.summary()


Model: "sensor_only"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sensor_input (InputLayer)            │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sensor_dropout (RandomSensorDropout) │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 224)                 │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout (MCDropout)               │ (None, 224)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 240)                 │          54,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout_1 (MCDropout)             │ (None, 240)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │             964 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,756 (221.70 KB)

 Trainable params: 56,756 (221.70 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
# View one batch of data from train_ds
for features, labels in train_ds.take(1):
    print("Feature batch shape:", features.shape)
    print("Label batch shape:", labels.shape)
    print("\nFirst 5 feature rows:\n", features[:5].numpy())
    print("\nFirst 5 labels:\n", labels[:5].numpy())


Feature batch shape: (32, 7)
Label batch shape: (32,)

First 5 feature rows:
 [[-0.73830456 -0.9110133  -1.5732417  -0.97052246  0.3734995   0.2683708
  -1.170219  ]
 [ 1.2744675   1.0383725   1.4626192   1.4248024   0.9388981   1.0232955
   1.1512552 ]
 [ 0.7040027   0.952998    0.02552526  0.16060321  0.5058269   0.42730236
   0.3165678 ]
 [-1.4594582  -1.480177   -0.8546948  -0.14990187  0.20508291  0.20214933
  -1.4962687 ]
 [-0.18936673 -0.3845368   0.9596363   0.6041819  -0.14378004 -0.73157346
   0.3296098 ]]

First 5 labels:
 [3 1 1 3 0]


In [12]:
# Define a ModelCheckpoint callback to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_sensor_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model using tf.data.Dataset objects
history = sensor_model.fit(
    train_sen_ds,
    validation_data=val_sen_ds,
    epochs=100,
    callbacks=[checkpoint]
)


Epoch 1/100


I0000 00:00:1746396619.122317      77 service.cc:148] XLA service 0x7823a8002d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746396619.124102      77 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746396619.124122      77 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746396619.435679      77 cuda_dnn.cc:529] Loaded cuDNN version 90300


109/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6604 - loss: 0.8611

I0000 00:00:1746396621.348802      77 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Epoch 1: val_accuracy improved from -inf to 0.84531, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6822 - loss: 0.7964 - val_accuracy: 0.8453 - val_loss: 0.2715
Epoch 2/100
126/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8168 - loss: 0.3860
Epoch 2: val_accuracy improved from 0.84531 to 0.90391, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8180 - loss: 0.3836 - val_accuracy: 0.9039 - val_loss: 0.2254
Epoch 3/100
124/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8444 - loss: 0.3450
Epoch 3: val_accuracy improved from 0.90391 to 0.92344, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8449 - loss: 0.3438 - val_accuracy: 0.9234 - val_loss: 0.1941
Epoch 4/100
126/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8519 - loss: 0.3092
Epoch 4: val_accuracy did not improve from 0.92344
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accurac

In [13]:
#testing with monte carlo or soemthing??

def predict_mc(model, dataset, T=50):
    all_preds = []
    all_labels = []
    for sens_batch, lbl_batch in dataset:
        preds_t = [model(sens_batch).numpy() for _ in range(T)]
        preds_t = np.stack(preds_t, axis=0)  # (T, batch, classes)
        mean_preds = preds_t.mean(axis=0)    # (batch, classes)
        all_preds.append(mean_preds)
        all_labels.append(lbl_batch.numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    y_pred = np.argmax(all_preds, axis=1)
    return np.mean(y_pred == all_labels)

# Compute test accuracy using MC dropout
sensor_model_test_acc = predict_mc(sensor_model,  test_sen_ds, T=50)
print(f"Sensor model MC dropout test accuracy: {sensor_model_test_acc:.4f}")

Sensor model MC dropout test accuracy: 0.9547


In [14]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

# 1. Save model as .h5 to Kaggle outputs directory
model_save_path = '/kaggle/working/best_sensor_model.h5'
sensor_model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

# 2. Evaluate on test set
# Get true labels and predictions
y_true = []
y_pred = []

for batch_x, batch_y in test_sen_ds:
    preds = sensor_model.predict(batch_x)
    y_true.extend(batch_y.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# 3. Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 4. Classification metrics
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# 5. Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Model saved to: /kaggle/working/best_sensor_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 854ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       160
           1       0.90      0.91      0.91       160

In [16]:
# Define a ModelCheckpoint callback to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_sensor_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model using tf.data.Dataset objects
history = sensor_model.fit(
    train_sen_ds,
    validation_data=val_sen_ds,
    epochs=100,
    callbacks=[checkpoint]
)
#testing with monte carlo or soemthing??

def predict_mc(model, dataset, T=50):
    all_preds = []
    all_labels = []
    for sens_batch, lbl_batch in dataset:
        preds_t = [model(sens_batch).numpy() for _ in range(T)]
        preds_t = np.stack(preds_t, axis=0)  # (T, batch, classes)
        mean_preds = preds_t.mean(axis=0)    # (batch, classes)
        all_preds.append(mean_preds)
        all_labels.append(lbl_batch.numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    y_pred = np.argmax(all_preds, axis=1)
    return np.mean(y_pred == all_labels)

# Compute test accuracy using MC dropout
sensor_model_test_acc = predict_mc(sensor_model,  test_sen_ds, T=50)
print(f"Sensor model MC dropout test accuracy: {sensor_model_test_acc:.4f}")

Epoch 1/100
121/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3235 - loss: 1.3511
Epoch 1: val_accuracy improved from -inf to 0.43594, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3261 - loss: 1.3474 - val_accuracy: 0.4359 - val_loss: 1.2112
Epoch 2/100
120/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4158 - loss: 1.2293
Epoch 2: val_accuracy improved from 0.43594 to 0.52500, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4181 - loss: 1.2253 - val_accuracy: 0.5250 - val_loss: 1.0431
Epoch 3/100
122/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4844 - loss: 1.1302
Epoch 3: val_accuracy improved from 0.52500 to 0.54844, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4857 - loss: 1.1268 - val_accuracy: 0.5484 - val_loss: 0.9648
Epoch 4/100
121/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5258 - loss: 1.0437
Epoch 4: val_acc

In [15]:
#without genetic hyperparamter tuning

from models import build_sensor_model, MCDropout, RandomSensorDropout  # Import necessary components
#dropout=0.036, hidden=224,240, lr=7.07e-04
# Build the sensor-only model
sensor_model = build_sensor_model(
   input_dim=7,
    sensor_dropout_rate=0.3,
    layer_dropout = 0.5,
    hidden_units=(64, 32),
    output_units=4,
    lr=1e-4
)

# Display model architecture
sensor_model.summary()


Model: "sensor_only"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sensor_input (InputLayer)            │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sensor_dropout (RandomSensorDropout) │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout_2 (MCDropout)             │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout_3 (MCDropout)             │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,724 (10.64 KB)

 Trainable params: 2,724 (10.64 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

from data_loader import load_and_split_image_data
from models import build_image_model

# --- Paths & Parameters ---
IMG_DIR = "/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images"
IMG_SIZE = (120, 160)
BATCH_SIZE = 32
EPOCHS = 30

# --- Load image-only dataset ---
train_ds, val_ds, test_ds = load_and_split_image_data(
    data_dir=IMG_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


In [18]:
# --- Build model with correct argument ---
model = build_image_model(img_shape=(120, 160, 3), output_units=4)

# --- Callbacks ---
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_image_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# --- Train model ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[checkpoint]
)


Epoch 1/50
137/140 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2829 - loss: 1.3870
Epoch 1: val_accuracy improved from -inf to 0.38203, saving model to best_image_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 39s 67ms/step - accuracy: 0.2842 - loss: 1.3865 - val_accuracy: 0.3820 - val_loss: 1.3292
Epoch 2/50
136/140 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3778 - loss: 1.3187
Epoch 2: val_accuracy improved from 0.38203 to 0.41953, saving model to best_image_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 51ms/step - accuracy: 0.3786 - loss: 1.3181 - val_accuracy: 0.4195 - val_loss: 1.2687
Epoch 3/50
137/140 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4427 - loss: 1.2468
Epoch 3: val_accuracy improved from 0.41953 to 0.47891, saving model to best_image_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 50ms/step - accuracy: 0.4429 - loss: 1.2465 - val_accuracy: 0.4789 - val_loss: 1.2154
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4918 - loss: 1.1822
Epoch 4: val_

In [21]:
# --- Save as .h5 to Kaggle output ---
model.save("/kaggle/working/best_image_model.h5")

# --- Predict on test set ---
y_true, y_pred = [], []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x)
    y_true.extend(batch_y.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# --- Evaluation ---
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       160
           1       0.53      0.30      0.38       160
           2       0.61      0.77      0.68       160

In [8]:
def load_and_split_multimodal_data(
    sensor_csv_path: str,
    image_data_dir: str,
    test_size: float = 0.1,
    val_size: float = 0.2,
    batch_size: int = 32,
    img_size: tuple = (120, 160),
    random_state: int = 42
):
    """
    Load and split both sensor CSV and image folders into multimodal tf.data.Datasets.

    Args:
        sensor_csv_path: Path to the gas sensor CSV.
        image_data_dir: Base directory of image class subfolders.
        test_size: Fraction of data reserved for testing.
        val_size: Fraction of remaining data reserved for validation.
        batch_size: Batch size for tf.data pipelines.
        img_size: Size to resize images to (height, width).
        random_state: Seed for reproducibility.

    Returns:
        train_ds, val_ds, test_ds: tf.data.Dataset yielding ((sensor, image), label).
    """
    # 1) Load sensor CSV
    sdf = pd.read_csv(sensor_csv_path)
    sdf = sdf.drop(columns=["Serial Number"], errors='ignore')
    sdf['Gas'] = sdf['Gas'].astype('category').cat.codes

    # Build image filename column
    sdf['Image_File'] = sdf['Corresponding Image Name'].astype(str) + ".png"

    # Extract sensor features and labels
    sensor_cols = [c for c in sdf.columns if c not in ['Gas', 'Corresponding Image Name', 'Image_File']]
    sensors = sdf[sensor_cols].values.astype('float32')
    labels = sdf['Gas'].values.astype('int32')

    # Normalize sensors
    scaler = StandardScaler().fit(sensors)
    sensors = scaler.transform(sensors)

    # Map image names to full paths
    base = image_data_dir
    def find_path(fname):
        for cls in os.listdir(base):
            p = os.path.join(base, cls, fname)
            if os.path.exists(p):
                return p
        return None

    paths = np.array(sdf['Image_File'].map(find_path))
    valid = ~pd.isna(paths)

    sensors = sensors[valid]
    paths   = paths[valid]
    labels  = labels[valid]

    # Split multimodal arrays
    X_temp, X_test, S_temp, S_test, y_temp, y_test = train_test_split(
        paths, sensors, labels,
        test_size=test_size, stratify=labels, random_state=random_state
    )
    val_frac = val_size / (1 - test_size)
    X_train, X_val, S_train, S_val, y_train, y_val = train_test_split(
        X_temp, S_temp, y_temp,
        test_size=val_frac, stratify=y_temp, random_state=random_state
    )

    # Define loader
    def loader(path, sens, lab):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        sens = tf.cast(sens, tf.float32)
        return (sens, img), lab

    # Build tf.data datasets
    def make_ds(paths, sensors, labels, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((paths, sensors, labels))
        ds = ds.map(loader, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(buffer_size=len(labels), seed=random_state)
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_ds(X_train, S_train, y_train, shuffle=True)
    val_ds   = make_ds(X_val,   S_val,   y_val,   shuffle=False)
    test_ds  = make_ds(X_test,  S_test,  y_test,  shuffle=False)

    return train_ds, val_ds, test_ds

In [9]:
import pandas as pd
import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Example call to load_and_split_multimodal_data and store results
sensor_csv_path = "/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Gas Sensors Measurements/Gas_Sensors_Measurements.csv"  # Path to your sensor CSV file
image_data_dir = "/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/dataset/Thermal Camera Images"  # Path to your image data directory

# Call the function and store the datasets
train_ds, val_ds, test_multi_ds = load_and_split_multimodal_data(
    sensor_csv_path=sensor_csv_path,
    image_data_dir=image_data_dir,
    test_size=0.1,
    val_size=0.2,
    batch_size=32,
    img_size=(120, 160),
    random_state=42
)

# Now train_ds, val_ds, and test_ds are ready for use in your model training and evaluation.


In [7]:

import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Add the 'src' folder to Python path
sys.path.append("/kaggle/input/multimodal/Multimodal-Gas-Leakage-Detection-main/src")

from models import RandomSensorDropout, MCDropout # Only sensor model
# Path to your saved models
sensor_model_path = "/kaggle/input/sensormodel/keras/default/1/best_sensor_model.h5"
image_model_path = "/kaggle/input/imagemodel/keras/default/1/best_image_model.h5"


sensor_model = tf.keras.models.load_model(
    "/kaggle/input/sensormodel/keras/default/1/best_sensor_model.h5",
    custom_objects={
        'RandomSensorDropout': RandomSensorDropout,
        'MCDropout': MCDropout
    },
    compile=False
)

# Load models
# Load model with custom layer

#image_model = tf.keras.models.load_model(image_model_path, compile=False)

# # Optionally compile them if you want to use them for evaluation or further training
# sensor_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# image_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [14]:
sensor_model.summary()

Model: "sensor_only"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sensor_input (InputLayer)            │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sensor_dropout (RandomSensorDropout) │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 224)                 │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout (MCDropout)               │ (None, 224)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 240)                 │          54,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout_1 (MCDropout)             │ (None, 240)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │             964 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,756 (221.70 KB)

 Trainable params: 56,756 (221.70 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
def build_image_model(
    img_shape=(120,160,3),
    conv_filters=(32, 64 , 128),
    layer_dropout = 0.5,
    dense_units=64,
    output_units=4,
    lr=1e-4
):
    """
    Image-only CNN classifier.
    """
    inp = tf.keras.Input(shape=img_shape, name="image_input")
    x = tf.keras.layers.Conv2D(conv_filters[0], 3, activation="relu", padding="same", use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[1], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[2], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = MCSpatialDropout2D(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(dense_units, activation="relu")(x)
    x = MCDropout(layer_dropout)(x)
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(x)

    model = tf.keras.Model(inp, out, name="image_only")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [29]:
# # Load image model with the same custom layers
# image_model = tf.keras.models.load_model(
#     "/kaggle/input/imagemodel/keras/default/1/best_image_model.h5",
#     custom_objects={
#         'RandomSensorDropout': RandomSensorDropout,
#         'MCDropout': MCDropout
#     },
#     compile=False
# )

class MCSpatialDropout2D(tf.keras.layers.SpatialDropout2D):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference 
        return super().call(inputs, training=True)
# Step 1: Rebuild the model architecture
image_model = build_image_model(img_shape=(120, 160, 3), output_units=4)  # Or however you originally defined it
image_model.load_weights('/kaggle/input/model_mc_image/keras/default/1/model_mc_.weights.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
image_model.summary()

Model: "image_only"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)             │ (None, 120, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 120, 160, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 120, 160, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 60, 80, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 60, 80, 64)          │          18,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 60, 80, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 40, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 30, 40, 128)         │          73,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 30, 40, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_spatial_dropout2d_1               │ (None, 30, 40, 128)         │               0 │
│ (MCSpatialDropout2D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mc_dropout_1 (MCDropout)             │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 102,436 (400.14 KB)

 Trainable params: 101,988 (398.39 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
#BELOW IS WITH MONTE CARLO DROPOUT FOR IMAGE AND SENSOR 

In [35]:
def predict_with_late_fusion(image_model, sensor_model, test_ds, weight_image=0.5, weight_sensor=0.5):
    fused_preds = []
    true_labels = []

    for (sensor_batch, image_batch), label_batch in test_multi_ds:
        # Predict from both models
        preds_image = image_model(image_batch, training=False).numpy()
        preds_sensor = sensor_model(sensor_batch, training=False).numpy()

        # Weighted average of predictions
        fused = (weight_image * preds_image) + (weight_sensor * preds_sensor)

        fused_preds.append(fused)
        true_labels.append(label_batch.numpy())

    # Stack and compute accuracy
    fused_preds = np.concatenate(fused_preds, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    final_preds = np.argmax(fused_preds, axis=1)

    accuracy = np.mean(final_preds == true_labels)
    return accuracy
#test 1 with 0.3 weight to image and 0.7 to sensor
accuracy = predict_with_late_fusion(image_model, sensor_model, test_multi_ds, weight_image=0.5, weight_sensor=0.5)
print(f"Late Fusion Accuracy with weighted average: {accuracy:.4f}")

Late Fusion Accuracy with weighted average: 0.9906


In [36]:
#late fusion another technique

def majority_vote_fusion(image_model, sensor_model, dataset):
    all_preds = []
    all_labels = []

    for (sensor_batch, image_batch), label_batch in test_multi_ds:
        pred_image = image_model(image_batch, training=False).numpy()
        pred_sensor = sensor_model(sensor_batch, training=False).numpy()

        class_image = np.argmax(pred_image, axis=1)
        class_sensor = np.argmax(pred_sensor, axis=1)

        final_preds = []
        for i in range(len(class_image)):
            if class_image[i] == class_sensor[i]:
                final_preds.append(class_image[i])  # same prediction
            else:
                # tie-break by higher confidence
                if np.max(pred_sensor[i]) > np.max(pred_image[i]):
                    final_preds.append(class_sensor[i])
                else:
                    final_preds.append(class_image[i])

        all_preds.extend(final_preds)
        all_labels.extend(label_batch.numpy())

    return np.mean(np.array(all_preds) == np.array(all_labels))
accuracy_majority = majority_vote_fusion(image_model, sensor_model, test_multi_ds)
print("Late Fusion Accuracy (Majority Voting):", accuracy_majority)


Late Fusion Accuracy (Majority Voting): 0.9890625


In [37]:
def logits_average_fusion(image_model, sensor_model, dataset):
    all_preds = []
    all_labels = []

    for (sensor_batch, image_batch), label_batch in test_multi_ds:
        logits_image = image_model(image_batch, training=False).numpy()
        logits_sensor = sensor_model(sensor_batch, training=False).numpy()

        combined_logits = 0.2 * logits_image + 0.8 * logits_sensor
        final_pred = np.argmax(combined_logits, axis=1)

        all_preds.extend(final_pred)
        all_labels.extend(label_batch.numpy())

    return np.mean(np.array(all_preds) == np.array(all_labels))
accuracy_logits_avg = logits_average_fusion(image_model, sensor_model, test_multi_ds)
print("Late Fusion Accuracy (Logits Averaging):", accuracy_logits_avg)


Late Fusion Accuracy (Logits Averaging): 0.9640625


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

def extract_features(model_img, model_sens, dataset):
    features = []
    labels = []

    for (sens_batch, img_batch), label_batch in test_multi_ds:
        preds_img  = model_img(img_batch, training=False).numpy()
        preds_sens = model_sens(sens_batch, training=False).numpy()
        combined = np.concatenate([preds_img, preds_sens], axis=1)
        features.append(combined)
        labels.append(label_batch.numpy())

    return np.concatenate(features), np.concatenate(labels)

# Step 1: Extract features from validation set
X_val_meta, y_val_meta = extract_features(image_model, sensor_model, val_ds)

# Step 2: Train meta-classifier
meta_clf = LogisticRegression(max_iter=1000)
meta_clf.fit(X_val_meta, y_val_meta)

# Step 3: Evaluate on test set
X_test_meta, y_test_meta = extract_features(image_model, sensor_model, test_multi_ds)
y_pred_meta = meta_clf.predict(X_test_meta)
accuracy_meta = accuracy_score(y_test_meta, y_pred_meta)

print("Late Fusion Accuracy (Meta-classifier):", accuracy_meta)


Late Fusion Accuracy (Meta-classifier): 0.990625


In [7]:
#WITHOUT MONTECARLOO 
sensor_model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sensor_dropout (RandomSensorDropout) │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,174 (31.93 KB)

 Trainable params: 2,724 (10.64 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,450 (21.29 KB)

In [10]:
import tensorflow as tf

def build_image_model(
    img_shape=(120,160,3),
    conv_filters=(32, 64 , 128),
    layer_dropout=0.5,
    dense_units=64,
    output_units=4,
    lr=1e-4
):
    """
    Image-only CNN classifier (standard Dropout version).
    """
    inp = tf.keras.Input(shape=img_shape, name="image_input")
    
    x = tf.keras.layers.Conv2D(conv_filters[0], 3, activation="relu", padding="same", use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    
    x = tf.keras.layers.Conv2D(conv_filters[1], 3, activation="relu", padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    
    x = tf.keras.layers.Conv2D(conv_filters[2], 3, activation="relu", padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)  # Standard SpatialDropout2D
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    x = tf.keras.layers.Dense(dense_units, activation="relu")(x)
    x = tf.keras.layers.Dropout(layer_dropout)(x)  # Standard Dropout
    
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(x)

    model = tf.keras.Model(inp, out, name="image_only")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [11]:
image_model = build_image_model(img_shape=(120, 160, 3), output_units=4)  # Or however you originally defined it
image_model.load_weights('/kaggle/input/modelimage/keras/default/1/model.weights.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
def predict_with_late_fusion(image_model, sensor_model, test_ds, weight_image=0.5, weight_sensor=0.5):
    fused_preds = []
    true_labels = []

    for (sensor_batch, image_batch), label_batch in test_multi_ds:
        # Predict from both models
        preds_image = image_model(image_batch, training=False).numpy()
        preds_sensor = sensor_model(sensor_batch, training=False).numpy()

        # Weighted average of predictions
        fused = (weight_image * preds_image) + (weight_sensor * preds_sensor)

        fused_preds.append(fused)
        true_labels.append(label_batch.numpy())

    # Stack and compute accuracy
    fused_preds = np.concatenate(fused_preds, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    final_preds = np.argmax(fused_preds, axis=1)

    accuracy = np.mean(final_preds == true_labels)
    return accuracy
#test 1 with 0.3 weight to image and 0.7 to sensor
accuracy = predict_with_late_fusion(image_model, sensor_model, test_multi_ds, weight_image=0.5, weight_sensor=0.5)
print(f"Late Fusion Accuracy with weighted average: {accuracy:.4f}")

Late Fusion Accuracy with weighted average: 0.9891


In [18]:
#late fusion another technique

def majority_vote_fusion(image_model, sensor_model, dataset):
    all_preds = []
    all_labels = []

    for (sensor_batch, image_batch), label_batch in test_multi_ds:
        pred_image = image_model(image_batch, training=False).numpy()
        pred_sensor = sensor_model(sensor_batch, training=False).numpy()

        class_image = np.argmax(pred_image, axis=1)
        class_sensor = np.argmax(pred_sensor, axis=1)

        final_preds = []
        for i in range(len(class_image)):
            if class_image[i] == class_sensor[i]:
                final_preds.append(class_image[i])  # same prediction
            else:
                # tie-break by higher confidence
                if np.max(pred_sensor[i]) > np.max(pred_image[i]):
                    final_preds.append(class_sensor[i])
                else:
                    final_preds.append(class_image[i])

        all_preds.extend(final_preds)
        all_labels.extend(label_batch.numpy())

    return np.mean(np.array(all_preds) == np.array(all_labels))
accuracy_majority = majority_vote_fusion(image_model, sensor_model, test_multi_ds)
print("Late Fusion Accuracy (Majority Voting):", accuracy_majority)


Late Fusion Accuracy (Majority Voting): 0.9890625


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

def extract_features(model_img, model_sens, dataset):
    features = []
    labels = []

    for (sens_batch, img_batch), label_batch in test_multi_ds:
        preds_img  = model_img(img_batch, training=False).numpy()
        preds_sens = model_sens(sens_batch, training=False).numpy()
        combined = np.concatenate([preds_img, preds_sens], axis=1)
        features.append(combined)
        labels.append(label_batch.numpy())

    return np.concatenate(features), np.concatenate(labels)

# Step 1: Extract features from validation set
X_val_meta, y_val_meta = extract_features(image_model, sensor_model, val_ds)

# Step 2: Train meta-classifier
meta_clf = LogisticRegression(max_iter=1000)
meta_clf.fit(X_val_meta, y_val_meta)

# Step 3: Evaluate on test set
X_test_meta, y_test_meta = extract_features(image_model, sensor_model, test_multi_ds)
y_pred_meta = meta_clf.predict(X_test_meta)
accuracy_meta = accuracy_score(y_test_meta, y_pred_meta)

print("Late Fusion Accuracy (Meta-classifier):", accuracy_meta)


Late Fusion Accuracy (Meta-classifier): 0.990625


In [4]:
class RandomSensorDropout(tf.keras.layers.Layer):
    """
    Layer that randomly zeros individual sensor channels with a given rate during training.
    Supports proper serialization.
    """
    def __init__(self, rate=0.3, **kwargs):
        super().__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=False):
        if training and self.rate > 0.0:
            mask = tf.cast(tf.random.uniform(tf.shape(inputs)) > self.rate, inputs.dtype)
            return inputs * mask
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"rate": self.rate})
        return config
    

def build_sensor_model(input_dim=7,
                       sensor_dropout_rate=0.3,
                       dropout_rate=0.5,
                       hidden_units=(64, 32),
                       output_units=4,
                       lr=1e-4):
    inp = tf.keras.layers.Input(shape=(input_dim,))
    x = RandomSensorDropout(sensor_dropout_rate, name="sensor_dropout")(inp)
    x = tf.keras.layers.Dense(hidden_units[0], activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(hidden_units[1], activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    out = tf.keras.layers.Dense(output_units, activation='softmax')(x)

    model = tf.keras.Model(inputs=inp, outputs=out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [15]:
#without genetic hyperparamter tuning

#from models import build_sensor_model, MCDropout, RandomSensorDropout  # Import necessary components
#dropout=0.036, hidden=224,240, lr=7.07e-04
# Build the sensor-only model
sensor_model = build_sensor_model(
   input_dim=7,
    sensor_dropout_rate=0.036,
    dropout_rate = 0.3,
    hidden_units=(224, 240),
    output_units=4,
    lr=7.07e-4
)

# Display model architecture
sensor_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sensor_dropout (RandomSensorDropout) │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 224)                 │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 224)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 240)                 │          54,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 240)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4)                   │             964 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,756 (221.70 KB)

 Trainable params: 56,756 (221.70 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Define a ModelCheckpoint callback to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_sensor_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model using tf.data.Dataset objects
history = sensor_model.fit(
    train_sen_ds,
    validation_data=val_sen_ds,
    epochs=100,
    callbacks=[checkpoint]
)
#testing with monte carlo or soemthing??


Epoch 1/100
120/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7111 - loss: 0.7271
Epoch 1: val_accuracy improved from -inf to 0.90859, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7261 - loss: 0.6864 - val_accuracy: 0.9086 - val_loss: 0.2211
Epoch 2/100
125/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8900 - loss: 0.2530
Epoch 2: val_accuracy improved from 0.90859 to 0.93125, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8907 - loss: 0.2519 - val_accuracy: 0.9312 - val_loss: 0.1841
Epoch 3/100
121/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9060 - loss: 0.2133
Epoch 3: val_accuracy improved from 0.93125 to 0.93359, saving model to best_sensor_model.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9062 - loss: 0.2129 - val_accuracy: 0.9336 - val_loss: 0.1645
Epoch 4/100
120/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9066 - loss: 0.2084
Epoch 4: val_acc

Sensor model MC dropout test accuracy: 0.8719
